In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [10]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:                \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS   PPL     NLL   
5000     56   28   99  593.52  831  590.12  3.27  0.14  602.60  1249  598.64   
10000    54   57   24  593.70  822  589.84  3.74  0.12  579.85  1000  575.76   
15000    54   85  124  593.15  822  589.08  3.97  0.10  574.77   922  570.60   
20000    55  114   49  593.64  822  589.45  4.11  0.09  571.78   881  567.40   
25000    54  142  149  593.39  822  589.12  4.20  0.08  570.77   879  566.38   
30000    53  171   74  593.24  820  588.91  4.26  0.08  569.96   856  565.53   
35000    53  199  174  593.41  819  589.04  4.31  0.07  569.63   866  565.17   
40000    55  228   99  593.01  818  588.61  4.34  0.07  569.47   851  564.99   
45000    54  257   24  593.12  817  588.71  4.37  0.06  569.19   853  564.67   
50000    54  285  124  592.83  815  588.39  4.39  0.06  569.46   855  564.99   
55000    55  314   49  592.91  814  588.45  4.42  0.06  568.91   846  564.45   
60000    55  342  149  592.77  813  588.30  4.43  0.05  569.04   850  564.54   
65000    55  371   74  592.68  812  588.20  4.45  0.05  568.55   839  564.07   
70000    54  399  174  592.75  812  588.27  4.46  0.05  568.78   848  564.31   
75000    53  428   99  592.55  811  588.05  4.47  0.05  568.67   840  564.19   
80000    53  457   24  592.61  810  588.11  4.48  0.05  568.40   841  563.89   
85000    54  485  124  592.44  809  587.94  4.49  0.04  568.92   848  564.46   
90000    53  514   49  592.49  809  587.98  4.50  0.04  568.47   839  563.97   
95000    53  542  149  592.41  808  587.90  4.50  0.04  568.59   842  564.07   
100000   54  571   74  592.35  807  587.84  4.51  0.04  568.35   836  563.84   
105000   50  599  174  592.41  807  587.89  4.52  0.04  568.30   841  563.85   
110000   49  628   99  592.28  806  587.76  4.52  0.04  568.40   836  563.94   
115000   49  657   24  592.32  806  587.80  4.53  0.04  568.25   838  563.76   
120000   50  685  124  592.21  805  587.69  4.53  0.04  568.62   841  564.10   
125000   53  714   49  592.24  805  587.72  4.53  0.04  568.26   837  563.79   
130000   55  742  149  592.18  805  587.66  4.54  0.04  568.40   840  563.93   
135000   54  771   74  592.14  804  587.62  4.54  0.03  568.22   832  563.67   
140000   53  799  174  592.19  804  587.66  4.54  0.03  568.23   839  563.76   
145000   54  828   99  592.09  803  587.56  4.55  0.03  568.48   836  563.94   
150000   56  857   24  592.12  803  587.59  4.55  0.03  568.17   835  563.67   
155000   54  885  124  592.04  803  587.50  4.55  0.03  568.50   838  563.99   
160000   54  914   49  592.06  802  587.52  4.56  0.03  568.41   838  563.91   
165000   51  942  149  592.02  802  587.48  4.56  0.03  568.36   838  563.83   
170000   55  971   74  591.99  802  587.44  4.56  0.03  568.06   831  563.59   
175000   53  999  174  592.02  801  587.48  4.56  0.03  568.04   836  563.55   

                     TEST:       SPEC:             HIER:        
          KL   REG    LOSS   PPL     1     2     3 CHILD OTHER  
5000    3.89  0.07  602.33  1245  0.13  0.42  0.45  0.39  0.26  
10000   4.00  0.10  579.64   999  0.10  0.42  0.25  0.43  0.35  
15000   4.12  0.06  574.66   922  0.19  0.35  0.26  0.49  0.42  
20000   4.32  0.05  571.73   879  0.13  0.43  0.30  0.40  0.33  
25000   4.35  0.04  570.75   877  0.15  0.41  0.33  0.39  0.35  
30000   4.39  0.04  569.87   855  0.13  0.42  0.34  0.37  0.33  
35000   4.42  0.04  569.87   855  0.14  0.42  0.37  0.35  0.31  
40000   4.44  0.03  569.50   851  0.14  0.41  0.36  0.36  0.33  
45000   4.48  0.03  569.50   851  0.13  0.43  0.34  0.36  0.32  
50000   4.44  0.03  569.50   851  0.19  0.38  0.33  0.39  0.37  
55000   4.43  0.03  568.93   847  0.14  0.42  0.34  0.36  0.32  
60000   4.47  0.02  568.93   847  0.16  0.40  0.34  0.38  0.35  
65000   4.45  0.03  568.60   839  0.13  0.42  0.35  0.36  0.33  
70000   4.44  0.02  568.60   839  0.15  0.42  0.36  0.36  0.32  
75000   4.46  0.02  56

0 R: 1.000 P: 0.155 write use article key know one post get like anyone
   1 R: 0.311 P: 0.067 space launch research new use year university nasa center include
     11 R: 0.161 P: 0.161 god say one christian people write jesus think believe know
     12 R: 0.083 P: 0.083 people gun israel government state write law article israeli right
   2 R: 0.293 P: 0.057 go say president make think people get work know tax
     21 R: 0.174 P: 0.174 go get say write one article see know like think
     22 R: 0.061 P: 0.061 game team play player year go win hockey get season
   3 R: 0.242 P: 0.105 file use window program image version server available run get
     31 R: 0.028 P: 0.028 turkish armenian armenians people turks armenia turkey say greek government
     32 R: 0.109 P: 0.109 use drive one get work card problem system write like
